In [1]:
import time
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import torch

In [2]:
batch_size = 128
learning_rate = 1e-2
num_epochs = 20

In [3]:
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
def to_np(x):
    return x.cpu().data.numpy()

In [5]:
class CNN(nn.Module):
    def __init__(self, in_dim, n_class):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(in_dim, 6, 3, stride=1, padding=1),
                                 nn.ReLU(True),
                                 nn.MaxPool2d(2, 2),
                                 nn.Conv2d(6, 16, 5, stride=1, padding=0),
                                 nn.ReLU(True),
                                 nn.MaxPool2d(2, 2))
        
        self.fc = nn.Sequential(nn.Linear(400, 120),
                               nn.Linear(120, 84),
                               nn.Linear(84, n_class))
        
    def forward(self, x):
        out = self.conv(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out


In [6]:
model = CNN(1, 10)
use_gpu = torch.cuda.is_available()  # 判断是否有gpu
if use_gpu:
    model = model.cuda()
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    since = time.time()
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        
        if use_gpu:
            img = Variable(img).cuda()
            label = Variable(label).cuda()
        else:
            img = Variable(img)
            label = Variable(label)
            
        out = model(img)
        loss = criterion(out, label)  # 得到的是batch_size的均值
       
        running_loss += loss.data[0] * label.size(0)  # 因此乘以batch_size的值得到总得损失
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum().float()  # 记得将整数转换成浮点数
    
        running_acc += num_correct.data[0]
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 300 == 0:
            
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                epoch + 1, num_epochs, running_loss / (batch_size * i),
                running_acc / (batch_size * i)))
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, running_loss / (len(train_dataset)), running_acc / (len(
            train_dataset))))
    
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for data in test_loader:
        img, label = data

        if torch.cuda.is_available():
            img = Variable(img, volatile=True).cuda()
            label = Variable(label, volatile=True).cuda()
        else:
            img = Variable(img, volatile=True)
            label = Variable(label, volatile=True)
        out = model(img)
        loss = criterion(out, label)
        eval_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum().float()
        eval_acc += num_correct.data[0]
    print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
        test_dataset)), eval_acc / (len(test_dataset))))

/home/jiangxinyang848/anaconda3/envs/jiang/lib/python3.5/site-packages/ipykernel_launcher.py:26: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/jiangxinyang848/anaconda3/envs/jiang/lib/python3.5/site-packages/ipykernel_launcher.py:30: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[1/20] Loss: 2.300009, Acc: 0.119661
Finish 1 epoch, Loss: 2.296699, Acc: 0.125350


/home/jiangxinyang848/anaconda3/envs/jiang/lib/python3.5/site-packages/ipykernel_launcher.py:52: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/jiangxinyang848/anaconda3/envs/jiang/lib/python3.5/site-packages/ipykernel_launcher.py:53: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/jiangxinyang848/anaconda3/envs/jiang/lib/python3.5/site-packages/ipykernel_launcher.py:59: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/jiangxinyang848/anaconda3/envs/jiang/lib/python3.5/site-packages/ipykernel_launcher.py:62: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Test Loss: 2.286656, Acc: 0.177200
[2/20] Loss: 2.272280, Acc: 0.285703
Finish 2 epoch, Loss: 2.247938, Acc: 0.346800
Test Loss: 2.124559, Acc: 0.595700
[3/20] Loss: 1.313217, Acc: 0.718151
Finish 3 epoch, Loss: 1.017280, Acc: 0.769267
Test Loss: 0.417471, Acc: 0.883700
[4/20] Loss: 0.366531, Acc: 0.895052
Finish 4 epoch, Loss: 0.344128, Acc: 0.900617
Test Loss: 0.275626, Acc: 0.917100
[5/20] Loss: 0.265520, Acc: 0.921615
Finish 5 epoch, Loss: 0.253980, Acc: 0.924867
Test Loss: 0.210345, Acc: 0.936700
[6/20] Loss: 0.213107, Acc: 0.936693
Finish 6 epoch, Loss: 0.206868, Acc: 0.938667
Test Loss: 0.183137, Acc: 0.941800
[7/20] Loss: 0.179764, Acc: 0.946094
Finish 7 epoch, Loss: 0.175522, Acc: 0.947867
Test Loss: 0.150307, Acc: 0.952700
[8/20] Loss: 0.155130, Acc: 0.953958
Finish 8 epoch, Loss: 0.152707, Acc: 0.954350
Test Loss: 0.128027, Acc: 0.960400
[9/20] Loss: 0.137049, Acc: 0.958516
Finish 9 epoch, Loss: 0.136400, Acc: 0.958783
Test Loss: 0.113648, Acc: 0.964100
[10/20] Loss: 0.12786